In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
#from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
# from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing import sequence
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [25]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data = pd.read_csv('https://drive.google.com/file/d/1mqJzGqPJbLIKgdv1UOAKD6ACQTUEKgCo/view?usp=sharing/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1", engine="python", sep = '","')
# data.columns = ["label", "time", "date", "query", "username", "text"]

In [19]:
#df = pd.read_csv('https://drive.google.com/file/d/1rbPSejEsGZhcmGpDcoc1Xo1brPtEjia_/view?usp=sharing/twitter_cleaning_one.csv', encoding = "ISO-8859-1", engine="python", sep = ',', error_bad_lines=False)
df = pd.read_csv('twitter_cleaning_one.csv', engine="python", sep = ',', error_bad_lines=False)

In [20]:
df

,Unnamed: 0,label,text
0,0,0,"@switchfoot - awww, that's a bummer. you sh..."
1,1,0,is upset that he can't update his facebook by ...
2,2,0,@kenichan i dived many times for the ball. man...
3,3,0,my whole body feels itchy and like its on fire
4,4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...,...
1599995,1599995,1,just woke up. having no school is the best fee...
1599996,1599996,1,thewdb.com - very cool to hear old walt interv...
1599997,1599997,1,are you ready for your mojo makeover? ask me f...
1599998,1599998,1,happy 38th birthday to my boo of alll time!!! ...


In [30]:
df1 = df.iloc[0:10000]

In [22]:
df2 = df.iloc[-10000:-1]

In [23]:
df2

,Unnamed: 0,label,text
1590000,1590000,1,@michaelahills yep
1590001,1590001,1,need to do homework.. can't wait for lvatt.. i...
1590002,1590002,1,"@annesudworth gosh, that's hectic... tea? i'm ..."
1590003,1590003,1,@hana77 daily: 8am - 12 midnight
1590004,1590004,1,watching the staff in the posh soho face cream...
...,...,...,...
1599994,1599994,1,"@cliff_forster yeah, that does work better tha..."
1599995,1599995,1,just woke up. having no school is the best fee...
1599996,1599996,1,thewdb.com - very cool to hear old walt interv...
1599997,1599997,1,are you ready for your mojo makeover? ask me f...


In [31]:
frames = [df1, df2]
df1 = pd.concat(frames)

In [32]:
df1 = df1.drop('Unnamed: 0', axis = 1)
df1

,label,text
0,0,"@switchfoot - awww, that's a bummer. you sh..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599994,1,"@cliff_forster yeah, that does work better tha..."
1599995,1,just woke up. having no school is the best fee...
1599996,1,thewdb.com - very cool to hear old walt interv...
1599997,1,are you ready for your mojo makeover? ask me f...


In [33]:
df1.reset_index(drop = True, inplace=True)
df1

,label,text
0,0,"@switchfoot - awww, that's a bummer. you sh..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
19994,1,"@cliff_forster yeah, that does work better tha..."
19995,1,just woke up. having no school is the best fee...
19996,1,thewdb.com - very cool to hear old walt interv...
19997,1,are you ready for your mojo makeover? ask me f...


In [44]:
nltk.download('stopwords')
stopword = stopwords.words('english')
pd.Series(stopword).count()

[nltk_data] Downloading package stopwords to C:\Users\Diana
[nltk_data]     Jaffé\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [48]:
# STOPWORD

df1['text'] = df1['text'].apply(lambda x: [item for item in x.split() if item not in stopword])

In [49]:
df1

,label,text
0,0,"[@switchfoot, -, awww,, that's, bummer., shoul..."
1,0,"[upset, can't, update, facebook, texting, it....."
2,0,"[@kenichan, dived, many, times, ball., managed..."
3,0,"[whole, body, feels, itchy, like, fire]"
4,0,"[@nationwideclass, no,, behaving, all., i'm, m..."
...,...,...
19994,1,"[@cliff_forster, yeah,, work, better, waiting,..."
19995,1,"[woke, up., school, best, feeling, ever]"
19996,1,"[thewdb.com, -, cool, hear, old, walt, intervi..."
19997,1,"[ready, mojo, makeover?, ask, details]"


def get_tokenize():
    return RegexpTokenizer(r'\w+|#\w+|<\w+>|%\w+|[^\w\s]+').tokenize 

In [37]:
get_tokenize(df1.text)

AttributeError: 'Series' object has no attribute 'get_tokenize'